In [0]:
%sql
CREATE CATALOG ecommerce;


In [0]:
%sql
USE CATALOG ecommerce;


In [0]:
%sql
CREATE SCHEMA bronze;
CREATE SCHEMA silver;
CREATE SCHEMA gold;


In [0]:
%sql
CREATE VOLUME ecommerce.bronze.bronze_vol;
CREATE VOLUME ecommerce.silver.silver_vol;
CREATE VOLUME ecommerce.gold.gold_vol;


In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA bronze;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS events (
  event_id INT,
  user_id INT,
  event_type STRING,
  event_time TIMESTAMP
)
USING DELTA;


In [0]:
%sql
INSERT INTO events VALUES
(1, 101, 'click', current_timestamp()),
(2, 102, 'purchase', current_timestamp());


num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA silver;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS events
USING DELTA
AS
SELECT *
FROM ecommerce.bronze.events;


num_affected_rows,num_inserted_rows


In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA gold;

CREATE TABLE IF NOT EXISTS products (
  product_name STRING,
  revenue DOUBLE,
  conversion_rate DOUBLE,
  purchases INT
)
USING DELTA;


In [0]:
%sql
INSERT INTO products VALUES
('Laptop', 250000, 0.18, 45),
('Phone', 180000, 0.22, 60),
('Headphones', 50000, 0.10, 5);


num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SHOW TABLES IN ecommerce.bronze;
SHOW TABLES IN ecommerce.silver;
SHOW TABLES IN ecommerce.gold;


database,tableName,isTemporary
gold,products,false


In [0]:
%sql
DESCRIBE EXTENDED ecommerce.bronze.events;
DESCRIBE EXTENDED ecommerce.silver.events;
DESCRIBE EXTENDED ecommerce.gold.products;


col_name,data_type,comment
product_name,string,null
revenue,double,null
conversion_rate,double,null
purchases,int,null
,,
# Delta Statistics Columns,,
Column Names,"product_name, revenue, conversion_rate, purchases",
Column Selection Method,first-32,
,,
# Detailed Table Information,,


In [0]:
%sql
SHOW GROUPS;


name,directGroup
admins,null
users,null


In [0]:
%sql
CREATE GROUP analysts;
CREATE GROUP engineers;


In [0]:
%sql
ALTER GROUP analysts ADD USER `testwest1221@gmail.com`;
ALTER GROUP engineers ADD USER `testwest1221@gmail.com`;


In [0]:
%sql
SELECT current_user();


current_user()
testwest1221@gmail.com


In [0]:
%sql
GRANT USE CATALOG ON CATALOG ecommerce TO `testwest1221@gmail.com`;

GRANT USE SCHEMA ON SCHEMA ecommerce.gold TO `testwest1221@gmail.com`;

GRANT SELECT ON TABLE ecommerce.gold.products TO `testwest1221@gmail.com`;


In [0]:
%sql
SHOW GROUPS;


name,directGroup
admins,null
users,null
engineers,null
analysts,null


In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA gold;

CREATE OR REPLACE VIEW top_products AS
SELECT
  product_name,
  revenue,
  conversion_rate
FROM products
WHERE purchases > 10
ORDER BY revenue DESC
LIMIT 100;


In [0]:
%sql
GRANT SELECT ON VIEW ecommerce.gold.top_products TO `testwest1221@gmail.com`;
